In [1]:
#notebook to find the super hero(s) with 1 connection(the least connections)

In [1]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import *
from pyspark.sql.types import *

In [2]:
# creating a spark session 
spark = SparkSession.builder.appName("super hero").getOrCreate()

In [3]:
#creating a schema for the names dataset
schema = StructType([
    StructField("id", IntegerType(), True),
    StructField("name", StringType(), True)
])

In [4]:
#reading datasets to spark dataframes
names = spark.read.schema(schema).option("sep", " ").csv("///Marvel-names.txt")
connection_lines = spark.read.text("///Marvel-graph.txt")


In [5]:
#finding the connections
connections = connection_lines.withColumn("id", split(trim(col("value")), " ")[0]) \
    .withColumn("connections", size(split(trim(col("value")), " ")) - 1) \
    .groupBy("id").agg(sum("connections").alias("connections")).sort("connections")
connections.show(5)

+----+-----------+
|  id|connections|
+----+-----------+
|2139|          0|
|3490|          0|
|3489|          0|
| 577|          0|
| 467|          0|
+----+-----------+
only showing top 5 rows



In [6]:
# finding the minimum number of connections
minConnectionCount = connections.agg(min("connections")).first()[0]
print(minConnectionCount)

0


In [7]:
minConnections = connections.filter(col("connections") == minConnectionCount)
minConnections.show(5)

+----+-----------+
|  id|connections|
+----+-----------+
| 577|          0|
| 467|          0|
|2139|          0|
|3489|          0|
|3490|          0|
+----+-----------+
only showing top 5 rows



In [8]:
#joining the the dataframe with the names dataframes
minConnectionsWithNames = minConnections.join(names, "id")
print("The following characters have only" + str(minConnectionCount) + " connection(s):")
minConnectionsWithNames.select("name").show()

The following characters have only0 connection(s):
+--------------------+
|                name|
+--------------------+
|        BERSERKER II|
|              BLARE/|
|MARVEL BOY II/MARTIN|
|MARVEL BOY/MARTIN BU|
|      GIURESCU, RADU|
|       CLUMSY FOULUP|
|              FENRIS|
|              RANDAK|
|           SHARKSKIN|
|     CALLAHAN, DANNY|
|         DEATHCHARGE|
|                RUNE|
|         SEA LEOPARD|
|         RED WOLF II|
|              ZANTOR|
|JOHNSON, LYNDON BAIN|
|          LUNATIK II|
|                KULL|
|GERVASE, LADY ALYSSA|
+--------------------+

